## Valuation of FX Swaps

In [1]:
import os
from os.path import join
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy.interpolate import interp1d

### 讀取資料

In [2]:
path = os.getcwd()
src = join(path, 'source')
out = join(path, 'output')

position = pd.read_excel(join(src, 'swap position.xlsx'), sheet_name=None)

* `position` contains information of swap, treasury curve and USDTWD forward curve.
* [FX Forward Rates Historical Data](http://www.mecklai.com/Digital/MecklaiData/HistoryData_Forward?AspxAutoDetectCookieSupport=1)

### Set Valuation Date

In [3]:
ValuationDate = datetime(2017, 12, 29)

In [4]:
position['swap'].head(5)

,AssetType_L1,AssetType_L2,ModelType,Name,TradeDate,SettleDate,MaturityDate,BuyCCY,BuyNotional,SellCCY,SellNotional,ContractRate
0,Derivatives,Swap,USDTWD_Swap,C20170522003F,2017-05-22,2018-01-24,NaN,TWD,1482850000,USD,50000000,29.6570
1,Derivatives,Swap,USDTWD_Swap,C20170522005F,2017-05-22,2018-01-24,NaN,TWD,504169000,USD,17000000,29.6570
2,Derivatives,Swap,USDTWD_Swap,C20170524004F,2017-05-24,2018-01-25,NaN,TWD,1491640000,USD,50000000,29.8328
3,Derivatives,Swap,USDTWD_Swap,C20170531007F,2017-05-31,2018-01-04,NaN,TWD,476483200,USD,16000000,29.7802
4,Derivatives,Swap,USDTWD_Swap,C20170531008F,2017-05-31,2018-01-04,NaN,TWD,684944600,USD,23000000,29.7802


In [5]:
# 2006~2019
position['TW_Treasury_ZC'][[ValuationDate]]

,2017-12-29 00:00:00
0,0.003382
1,0.003781
2,0.004236
3,0.004784
4,0.005058
...,...
57,0.015019
58,0.015070
59,0.015120
60,0.015168


In [6]:
# 2009~2020
position['USDTWD_Forward'][[ValuationDate]]

,2017-12-29 00:00:00
0,29.9410
1,29.9403
2,29.9398
3,29.9392
4,29.9387
5,29.9381
6,29.9375
7,29.9369
8,29.9363
9,29.9357


### FX Swap Valuation Function

In [7]:
def FXswap(ValuationDate, SettleDate, fwd_curve, dis_curve, amt_cur1, amt_cur2):
    
    t = (SettleDate - ValuationDate).days/360
    ctr_rate = amt_cur1/amt_cur2
    
    # 遠期匯率
    fw_rate = fwd_curve(t)
    # 折現因子
    DF = np.exp(-dis_curve(t)*t)
    # 計算評價日之利差
    swap_point = fw_rate - ctr_rate
    # 計算收益
    payoff = swap_point * amt_cur2
    # 價值折現
    value = payoff * DF
    
    return fw_rate, DF, value

### Calculating Values of Holding Contracts

In [8]:
FXvalue = pd.DataFrame(columns=['SettleDate', 'Contract Rate', 'Forward Rate', 'Discount Factor', 'MTM'])

for i in range(len(position['swap'])):
    ### swap information
    info = position['swap'].loc[i,:]
    
    amt_cur1 = info['BuyNotional']
    amt_cur2 = info['SellNotional']
    
    SettleDate = info['SettleDate']
    
    ### forward curve
    fwd_curve = interp1d(position['USDTWD_Forward'].iloc[:, 0], position['USDTWD_Forward'].loc[:, ValuationDate])

    ### discount curve
    dis_curve = interp1d(position['TW_Treasury_ZC'].iloc[:, 0], position['TW_Treasury_ZC'].loc[:, ValuationDate], fill_value='extrapolate')

    fw_rate, DF, p = FXswap(ValuationDate, SettleDate, fwd_curve, dis_curve, amt_cur1, amt_cur2)
    
    FXvalue.loc[i, :] = SettleDate, amt_cur1/amt_cur2, fw_rate, DF, p

In [9]:
FXvalue.head(5)

,SettleDate,Contract Rate,Forward Rate,Discount Factor,MTM
0,2018-01-24 00:00:00,29.657,29.940393333333333,0.999758,1.41662e+07
1,2018-01-24 00:00:00,29.657,29.940393333333333,0.999758,4.81652e+06
2,2018-01-25 00:00:00,29.8328,29.94037,0.999748,5.37714e+06
3,2018-01-04 00:00:00,29.7802,29.94086,0.999946,2.57042e+06
4,2018-01-04 00:00:00,29.7802,29.94086,0.999946,3.69498e+06


In [10]:
sum(FXvalue['MTM'])

634337063.691812